In [1]:
import torch
from utils import intersection_over_union
from collections import Counter

In [2]:
cell_indices = torch.arange(7).repeat(8, 7, 10).unsqueeze(-1)
cell_indices.shape

torch.Size([8, 7, 70, 1])

In [3]:
torch.arange(7).unsqueeze(1).repeat(3,1,7)

tensor([[[0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2, 2],
         [3, 3, 3, 3, 3, 3, 3],
         [4, 4, 4, 4, 4, 4, 4],
         [5, 5, 5, 5, 5, 5, 5],
         [6, 6, 6, 6, 6, 6, 6]],

        [[0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2, 2],
         [3, 3, 3, 3, 3, 3, 3],
         [4, 4, 4, 4, 4, 4, 4],
         [5, 5, 5, 5, 5, 5, 5],
         [6, 6, 6, 6, 6, 6, 6]],

        [[0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2, 2],
         [3, 3, 3, 3, 3, 3, 3],
         [4, 4, 4, 4, 4, 4, 4],
         [5, 5, 5, 5, 5, 5, 5],
         [6, 6, 6, 6, 6, 6, 6]]])

In [4]:
torch.randn(7,1).repeat(3,1,7)

tensor([[[ 0.2576,  0.2576,  0.2576,  0.2576,  0.2576,  0.2576,  0.2576],
         [-0.4488, -0.4488, -0.4488, -0.4488, -0.4488, -0.4488, -0.4488],
         [-0.6407, -0.6407, -0.6407, -0.6407, -0.6407, -0.6407, -0.6407],
         [ 1.0914,  1.0914,  1.0914,  1.0914,  1.0914,  1.0914,  1.0914],
         [-0.2838, -0.2838, -0.2838, -0.2838, -0.2838, -0.2838, -0.2838],
         [-0.4532, -0.4532, -0.4532, -0.4532, -0.4532, -0.4532, -0.4532],
         [-1.9346, -1.9346, -1.9346, -1.9346, -1.9346, -1.9346, -1.9346]],

        [[ 0.2576,  0.2576,  0.2576,  0.2576,  0.2576,  0.2576,  0.2576],
         [-0.4488, -0.4488, -0.4488, -0.4488, -0.4488, -0.4488, -0.4488],
         [-0.6407, -0.6407, -0.6407, -0.6407, -0.6407, -0.6407, -0.6407],
         [ 1.0914,  1.0914,  1.0914,  1.0914,  1.0914,  1.0914,  1.0914],
         [-0.2838, -0.2838, -0.2838, -0.2838, -0.2838, -0.2838, -0.2838],
         [-0.4532, -0.4532, -0.4532, -0.4532, -0.4532, -0.4532, -0.4532],
         [-1.9346, -1.9346, -1.9346,

In [5]:
torch.randn(2,2).repeat([1,2],).shape

torch.Size([2, 4])

In [6]:
a = torch.randn(3,4,5)
b = torch.randn(3,4,5)
c, idx = torch.max(a, dim=1)
c, idx

(tensor([[ 0.3510, -0.1542,  1.3281,  0.7312,  0.7922],
         [ 2.3446,  0.7696,  0.2838,  0.6153,  0.2147],
         [ 1.7086,  1.0809,  0.6460,  1.9291,  1.0936]]),
 tensor([[0, 1, 0, 0, 1],
         [3, 2, 2, 2, 2],
         [1, 3, 0, 1, 2]]))

In [7]:
torch.randint(1,7, (1,3))

tensor([[1, 1, 5]])

In [8]:
torch.randint(1,70, (2,3,4)).repeat(2,2,3)

tensor([[[62, 21, 53, 52, 62, 21, 53, 52, 62, 21, 53, 52],
         [49, 62, 50,  9, 49, 62, 50,  9, 49, 62, 50,  9],
         [30, 50, 47, 18, 30, 50, 47, 18, 30, 50, 47, 18],
         [62, 21, 53, 52, 62, 21, 53, 52, 62, 21, 53, 52],
         [49, 62, 50,  9, 49, 62, 50,  9, 49, 62, 50,  9],
         [30, 50, 47, 18, 30, 50, 47, 18, 30, 50, 47, 18]],

        [[40, 62, 34, 10, 40, 62, 34, 10, 40, 62, 34, 10],
         [20, 42, 45, 66, 20, 42, 45, 66, 20, 42, 45, 66],
         [26, 12, 52,  5, 26, 12, 52,  5, 26, 12, 52,  5],
         [40, 62, 34, 10, 40, 62, 34, 10, 40, 62, 34, 10],
         [20, 42, 45, 66, 20, 42, 45, 66, 20, 42, 45, 66],
         [26, 12, 52,  5, 26, 12, 52,  5, 26, 12, 52,  5]],

        [[62, 21, 53, 52, 62, 21, 53, 52, 62, 21, 53, 52],
         [49, 62, 50,  9, 49, 62, 50,  9, 49, 62, 50,  9],
         [30, 50, 47, 18, 30, 50, 47, 18, 30, 50, 47, 18],
         [62, 21, 53, 52, 62, 21, 53, 52, 62, 21, 53, 52],
         [49, 62, 50,  9, 49, 62, 50,  9, 49, 62, 50

In [9]:
predictions = torch.randn(4,6,6, 30)

a = predictions[..., 20:21]

b = predictions[..., 25:26]

a.shape, b.shape

(torch.Size([4, 6, 6, 1]), torch.Size([4, 6, 6, 1]))

In [10]:
bbox1 = predictions[...,21:25]
bbox2 = predictions[...,26:30]
bbox1.shape

torch.Size([4, 6, 6, 4])

In [11]:
c = torch.stack([a,b], dim=0)
c.shape

torch.Size([2, 4, 6, 6, 1])

In [12]:
best_box = torch.argmax(c,dim=0)
best_box

tensor([[[[1],
          [1],
          [0],
          [1],
          [1],
          [1]],

         [[1],
          [0],
          [0],
          [1],
          [0],
          [1]],

         [[0],
          [0],
          [0],
          [0],
          [1],
          [1]],

         [[1],
          [0],
          [1],
          [1],
          [0],
          [1]],

         [[0],
          [1],
          [1],
          [0],
          [1],
          [1]],

         [[1],
          [0],
          [1],
          [1],
          [1],
          [1]]],


        [[[0],
          [0],
          [0],
          [1],
          [1],
          [1]],

         [[1],
          [0],
          [0],
          [1],
          [1],
          [1]],

         [[1],
          [1],
          [0],
          [0],
          [1],
          [0]],

         [[0],
          [0],
          [1],
          [0],
          [1],
          [1]],

         [[0],
          [0],
          [1],
          [0],
          [1],
   

In [13]:
best_boxes = (1 - best_box)*bbox1
best_boxes

tensor([[[[-0.0000e+00,  0.0000e+00, -0.0000e+00,  0.0000e+00],
          [-0.0000e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00],
          [-7.0694e-01,  3.3962e-01, -1.7189e+00,  1.9964e+00],
          [ 0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[-0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00],
          [ 1.4221e+00,  1.6896e-01, -1.5503e+00,  8.0065e-01],
          [-8.2897e-01,  2.5451e+00, -1.3153e+00, -1.8303e+00],
          [-0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00],
          [ 9.8206e-01,  3.6427e-01, -2.0842e+00,  2.2915e-01],
          [ 0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 2.0001e-01,  8.5256e-01,  1.3785e-02,  7.2579e-01],
          [-1.7110e-01,  2.1408e-01, -1.2755e+00, -7.2162e-01],
          [ 5.1291e-01, -1.6198e+00,  4.3604e-01,  5.6492e-01],
          [-2.2058e+00,  1.4456e-01,

In [14]:
best_boxes = (1 - best_box)*bbox1 + best_box*bbox2
best_boxes

tensor([[[[ 1.9249e+00,  6.6873e-01, -5.7902e-01,  5.2854e-01],
          [ 1.6056e+00,  5.3056e-01,  1.1848e+00,  5.1350e-01],
          [-7.0694e-01,  3.3962e-01, -1.7189e+00,  1.9964e+00],
          [-1.5915e+00,  6.1651e-01,  2.1445e+00,  1.4228e-01],
          [ 9.6250e-01,  2.6333e-01,  5.2580e-01,  5.4588e-01],
          [ 7.4428e-04, -8.6368e-01, -1.0676e-01, -5.8598e-01]],

         [[-2.2860e+00, -6.9827e-02, -3.0511e-01,  1.5173e+00],
          [ 1.4221e+00,  1.6896e-01, -1.5503e+00,  8.0065e-01],
          [-8.2897e-01,  2.5451e+00, -1.3153e+00, -1.8303e+00],
          [ 1.1486e+00,  9.0576e-01, -1.5958e+00,  1.4882e+00],
          [ 9.8206e-01,  3.6427e-01, -2.0842e+00,  2.2915e-01],
          [-2.0816e-01, -1.8935e+00,  4.7360e-01,  3.6703e-01]],

         [[ 2.0001e-01,  8.5256e-01,  1.3785e-02,  7.2579e-01],
          [-1.7110e-01,  2.1408e-01, -1.2755e+00, -7.2162e-01],
          [ 5.1291e-01, -1.6198e+00,  4.3604e-01,  5.6492e-01],
          [-2.2058e+00,  1.4456e-01,

In [15]:
a = torch.randint(1,2,(2,2,3,5))
a

tensor([[[[1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]],

         [[1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]]],


        [[[1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]],

         [[1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]]]])

In [16]:
a = torch.randn((8,49,6))
a[...,0] = a[...,0].long()
a[0,0,:]

tensor([ 0.0000,  2.2342, -0.8635, -0.7942,  0.6425,  0.5319])

In [17]:
for x in a:
    print([x.item() for x in a[0,0,:]])

[0.0, 2.2342076301574707, -0.8635495901107788, -0.7942245006561279, 0.642523467540741, 0.5319207310676575]
[0.0, 2.2342076301574707, -0.8635495901107788, -0.7942245006561279, 0.642523467540741, 0.5319207310676575]
[0.0, 2.2342076301574707, -0.8635495901107788, -0.7942245006561279, 0.642523467540741, 0.5319207310676575]
[0.0, 2.2342076301574707, -0.8635495901107788, -0.7942245006561279, 0.642523467540741, 0.5319207310676575]
[0.0, 2.2342076301574707, -0.8635495901107788, -0.7942245006561279, 0.642523467540741, 0.5319207310676575]
[0.0, 2.2342076301574707, -0.8635495901107788, -0.7942245006561279, 0.642523467540741, 0.5319207310676575]
[0.0, 2.2342076301574707, -0.8635495901107788, -0.7942245006561279, 0.642523467540741, 0.5319207310676575]
[0.0, 2.2342076301574707, -0.8635495901107788, -0.7942245006561279, 0.642523467540741, 0.5319207310676575]


In [18]:
cell_indices = torch.arange(7).repeat(8,4,1).unsqueeze(-1)
cell_indices

tensor([[[[0],
          [1],
          [2],
          [3],
          [4],
          [5],
          [6]],

         [[0],
          [1],
          [2],
          [3],
          [4],
          [5],
          [6]],

         [[0],
          [1],
          [2],
          [3],
          [4],
          [5],
          [6]],

         [[0],
          [1],
          [2],
          [3],
          [4],
          [5],
          [6]]],


        [[[0],
          [1],
          [2],
          [3],
          [4],
          [5],
          [6]],

         [[0],
          [1],
          [2],
          [3],
          [4],
          [5],
          [6]],

         [[0],
          [1],
          [2],
          [3],
          [4],
          [5],
          [6]],

         [[0],
          [1],
          [2],
          [3],
          [4],
          [5],
          [6]]],


        [[[0],
          [1],
          [2],
          [3],
          [4],
          [5],
          [6]],

         [[0],
          [1],
   

In [19]:
cell_indices.permute(0,2,1,3)

tensor([[[[0],
          [0],
          [0],
          [0]],

         [[1],
          [1],
          [1],
          [1]],

         [[2],
          [2],
          [2],
          [2]],

         [[3],
          [3],
          [3],
          [3]],

         [[4],
          [4],
          [4],
          [4]],

         [[5],
          [5],
          [5],
          [5]],

         [[6],
          [6],
          [6],
          [6]]],


        [[[0],
          [0],
          [0],
          [0]],

         [[1],
          [1],
          [1],
          [1]],

         [[2],
          [2],
          [2],
          [2]],

         [[3],
          [3],
          [3],
          [3]],

         [[4],
          [4],
          [4],
          [4]],

         [[5],
          [5],
          [5],
          [5]],

         [[6],
          [6],
          [6],
          [6]]],


        [[[0],
          [0],
          [0],
          [0]],

         [[1],
          [1],
          [1],
          [1]],

    

In [20]:
a = torch.randint(0,3, (4,2,5))
# b = torch.randint(0,3, (4,2,5))
a,b

(tensor([[[2, 0, 0, 2, 2],
          [0, 0, 0, 1, 1]],
 
         [[2, 1, 0, 1, 0],
          [1, 0, 2, 0, 1]],
 
         [[0, 1, 2, 0, 2],
          [1, 1, 2, 1, 0]],
 
         [[2, 1, 1, 0, 0],
          [0, 2, 2, 1, 0]]]),
 tensor([[[[ 6.3897e-01],
           [ 5.5869e-01],
           [-4.2504e-01],
           [-1.5391e+00],
           [-3.3922e-01],
           [ 5.1860e-02]],
 
          [[-7.0685e-01],
           [-9.7999e-01],
           [-2.9667e+00],
           [-2.7024e-01],
           [-1.7266e-01],
           [ 1.6396e+00]],
 
          [[ 4.1863e-01],
           [-2.5000e+00],
           [ 1.5359e-02],
           [-8.9402e-01],
           [ 2.3979e+00],
           [ 1.2274e-01]],
 
          [[-1.6086e-01],
           [-4.2709e-01],
           [-2.8559e-01],
           [ 6.6623e-01],
           [-2.1404e+00],
           [ 1.0730e+00]],
 
          [[-9.5526e-01],
           [ 1.0634e+00],
           [ 3.9659e-01],
           [-1.8486e+00],
           [ 1.0772e+00],
      

In [21]:
c = torch.stack((a[...,1:2],a[...,2:3]), dim=0)
d = torch.cat((a[...,1].unsqueeze(0),a[...,2].unsqueeze(0)), dim=0)
c.shape,d.shape

(torch.Size([2, 4, 2, 1]), torch.Size([2, 4, 2]))

In [22]:
torch.argmax(d, dim=0).unsqueeze(-1)

tensor([[[0],
         [0]],

        [[0],
         [1]],

        [[1],
         [1]],

        [[0],
         [0]]])

In [24]:
torch.argmax(c, dim=0)

tensor([[[0],
         [0]],

        [[0],
         [1]],

        [[1],
         [1]],

        [[0],
         [0]]])